# Groep 4
# Zonne energie
## Exploratie data/opschonen

In [1]:
import snowflake.connector
import seaborn as sns
from seaborn import distplot
import matplotlib.pyplot as plt
from matplotlib.pyplot import plot, draw, show
import numpy as np 
import pandas as pd
from pandas_profiling import ProfileReport
from pandas.plotting import scatter_matrix
%matplotlib inline

In [2]:
ctx = snowflake.connector.connect(
    user='Karan',
password='Test1234!',
account='ty73594.west-europe.azure')
cs = ctx.cursor()

ctx.cursor().execute("use role sysadmin")
ctx.cursor().execute("use warehouse COMPUTE_WH")
ctx.cursor().execute("use database NORTHPOOL")
ctx.cursor().execute("use schema ASSIGNMENTB")

#### production_solar tabel in een dataframe stoppen

In [3]:
try:
    sql = 'SELECT * FROM "production_solar";'
    cs.execute(sql)
    df = cs.fetch_pandas_all()
finally:
    cs.close()
ctx.close()

#### Data dictionary 

Variabele |Type |Meetniveau |Omschrijving |Numeriek of Categorisch
:---:|:---:|:---:|:---:|:---:
datetime_start_utc|TIMESTAMP|Interval|Start tijdsblok in UTC|nvt
datetime_end_utc|TIMESTAMP|Interval|Eind tijdsblok in UTC|nvt
datetime_start_nl|TIMESTAMP|Interval|Start tijdsblok in NL tijd|nvt
datetime_end_nl|TIMESTAMP|Interval|Eind tijdsblok in NL tijd|nvt
datetime_start_ntz|TIMESTAMP|Interval|Start tijdsblok zonder tijdzone|nvt
provider|STRING|Nominaal|Leverancier van de data|Categorisch
area_codev|STRING|Nominaal|Het specifieke gebied waar de productie in geleverd wordt/is|nvt
area_type|STRING|Nominaal|Tpe gebied|Categorisch
interval|STRING|Interval|Tijdsblok in minuten (ofwel datetime_end_utc - datetime_start_utc)|Categorisch
energy_source|STRING|Nominaal|Manier van energieopwekking|Categorisch
phase|STRING|Nominaal|Forect of Actual data|Categorisch
production_mw|FLOAT|Ratio|Productie in MW (voorspeld of geleverd)|Numeriek
datetime_modification_utc|TIMESTAMP|Interval|Tijdstip waarop de data provider de data beschikbaar heeft gesteld (mist bekend) in UTC|nvt
datetime_fetch|TIMESTAMP|Interval|Tijdstip waarop Northpool de data opgehaald heeft van de data provider in UTC|nvt
datetime_load_snowflake|TIMESTAMP|Interval|Tijstip dat de data voor het eerst in Snowflake is beland|nvt
datetime_load_businessvault|TIMESTAMP|Interval|Tijdstip dat de data in de business vault tabel is beland|nvt
datetime_load_reporting|TIMESTAMP|Interval|Tijdstip dat de data in de reporting tabel is beland (deze tabel)|nvt

#### Profiel report van de production_solar tabel

In [4]:
# profile = ProfileReport(df, title="Pandas Profiling Report")
# profile.to_file(output_file="production_solar_profiling_report.html") # report wordt opgeslagen als html file.

#### Controleren op Null waardes

In [5]:
df.isnull().sum()

datetime_start_utc                  0
datetime_end_utc                    0
datetime_start_nl                   0
datetime_end_nl                     0
datetime_start_ntz                  0
provider                            0
area_code                           0
area_type                           0
interval                            0
energy_source                       0
phase                               0
production_mw                     192
datetime_modification_utc      208025
datetime_fetch                      0
datetime_load_snowflake             0
datetime_load_businessvault         0
datetime_load_reporting             0
dtype: int64

In [6]:
df = df.drop(['datetime_start_nl', 'datetime_end_nl', 'datetime_start_ntz', 'area_type', 'energy_source'], axis=1)

We halen de kolommen datetime_start_nl, datetime_end_nl en datetime_start_ntz uit de dataframe omdat deze overbodig zijn. Voor onze analyse gebruiken we de datetime_start_utc en datetime_end_utc kolommen. De kolommen area_type en energy_source halen we eruit omdat deze maar een mogelijke waarde bevatten en geen invloed hebben in ons onderzoek. 

In [7]:
df = df.dropna(axis=0, subset=['production_mw'])

In 192 records is de production_mw niet inbegrepen. Gezien ons onderzoek zullen wij deze records verwijderen uit de dataset. Wij willen namelijk de production_mw tussen phase FC1 en A vergelijken. 

Ook valt op dat er 208025 records zonder datetime_modification_utc tussen zit. Bij een provider is niet altijd bekend wanneer zei precies de data voor de voorspelling leveren. In dat geval wordt de cel leeg gelaten. Echter voor onze deelvragen is dit geen probleem. Voor accuraatheid en betrouwbaarheid gebruiken we het niet. Bij snelheid worden de Null waardes niet meegenomen. 

# Data prepareren
Per area_code maken we een dataframe aan. Als een area_code meerdere providers heeft, splitsen we die ook op in providers. Dit doen we omdat analyse willen doen per gebied en provider. 

In [8]:
df_solar_hu = df[df.area_code=='HU'].sort_values('datetime_start_utc')
df_solar_nl = df[df.area_code=='NL'].sort_values('datetime_start_utc')
df_solar_de = df[df.area_code=='DE'].sort_values('datetime_start_utc')
df_solar_ro = df[df.area_code=='RO'].sort_values('datetime_start_utc')
df_solar_dk2 = df[df.area_code=='DK2'].sort_values('datetime_start_utc')
df_solar_dk1 = df[df.area_code=='DK1'].sort_values('datetime_start_utc')
df_solar_gb = df[df.area_code=='GB'].sort_values('datetime_start_utc')
df_solar_no2 = df[df.area_code=='NO2'].sort_values('datetime_start_utc')
df_solar_no1 = df[df.area_code=='NO1'].sort_values('datetime_start_utc')
df_solar_es = df[df.area_code=='ES'].sort_values('datetime_start_utc')
df_solar_fi = df[df.area_code=='FI'].sort_values('datetime_start_utc')
df_solar_pl = df[df.area_code=='PL'].sort_values('datetime_start_utc')
df_solar_ch = df[df.area_code=='CH'].sort_values('datetime_start_utc')
df_solar_sk = df[df.area_code=='SK'].sort_values('datetime_start_utc')
df_solar_itcs = df[df.area_code=='IT-CS'].sort_values('datetime_start_utc')
df_solar_itno = df[df.area_code=='IT-NO'].sort_values('datetime_start_utc')
df_solar_cz = df[df.area_code=='CZ'].sort_values('datetime_start_utc')
df_solar_si = df[df.area_code=='SI'].sort_values('datetime_start_utc')

df_solar_be = df[df.area_code=='BE'].sort_values('datetime_start_utc')
df_solar_be_elia = df_solar_be[df_solar_be.provider == 'ELIA'].sort_values('datetime_start_utc')
df_solar_be_entsoeftp = df_solar_be[df_solar_be.provider == 'ENTSOE_FTP'].sort_values('datetime_start_utc')

df_solar_at = df[df.area_code=='AT'].sort_values('datetime_start_utc')
df_solar_at_entsoeftp = df_solar_at[df_solar_at.provider == 'ENTSOE_FTP'].sort_values('datetime_start_utc')
df_solar_at_apg = df_solar_at[df_solar_at.provider == 'APG'].sort_values('datetime_start_utc')

df_solar_fr = df[df.area_code=='FR'].sort_values('datetime_start_utc')
df_solar_fr_rteeco2 = df_solar_fr[df_solar_fr.provider == 'RTE_ECO2'].sort_values('datetime_start_utc')
df_solar_fr_entsoeftp = df_solar_fr[df_solar_fr.provider == 'ENTSOE_FTP'].sort_values('datetime_start_utc')
df_solar_fr_rte = df_solar_fr[df_solar_fr.provider == 'RTE'].sort_values('datetime_start_utc')

### Dataframes mergen
Alle data is nu opgesplitst. We gaan elke dataframe nog een keer opsplitsen in twee verschillende dataframes op basis van 'phase' (FC1 & A). Eenmaal opgesplitst voegen (mergen) we ze weer bij elkaar op basis van 'datetime_start_utc'. Dit doen we zodat een record zowel de FC1 als A waarde bevat, waardoor we gemakkelijker berekeningen kunnen uitvoeren.  

In [9]:
df_solar_hu_fc1 = df_solar_hu[df_solar_hu.phase == 'FC1']
df_solar_hu_a = df_solar_hu[df_solar_hu.phase == 'A']
df_solar_hu = df_solar_hu_a.merge(df_solar_hu_fc1, left_on=df_solar_hu_a['datetime_start_utc'], right_on=df_solar_hu_fc1['datetime_start_utc'])

df_solar_nl_fc1 = df_solar_nl[df_solar_nl.phase == 'FC1']
df_solar_nl_a = df_solar_nl[df_solar_nl.phase == 'A']
df_solar_nl = df_solar_nl_a.merge(df_solar_nl_fc1, left_on=df_solar_nl_a['datetime_start_utc'], right_on=df_solar_nl_fc1['datetime_start_utc'])


df_solar_de_fc1 = df_solar_de[df_solar_de.phase == 'FC1']
df_solar_de_a = df_solar_de[df_solar_de.phase == 'A']
df_solar_de = df_solar_de_a.merge(df_solar_de_fc1, left_on=df_solar_de_a['datetime_start_utc'], right_on=df_solar_de_fc1['datetime_start_utc'])

df_solar_ro_fc1 = df_solar_ro[df_solar_ro.phase == 'FC1']
df_solar_ro_a = df_solar_ro[df_solar_ro.phase == 'A']
df_solar_ro = df_solar_ro_a.merge(df_solar_ro_fc1, left_on=df_solar_ro_a['datetime_start_utc'], right_on=df_solar_ro_fc1['datetime_start_utc'])

df_solar_dk2_fc1 = df_solar_dk2[df_solar_dk2.phase == 'FC1']
df_solar_dk2_a = df_solar_dk2[df_solar_dk2.phase == 'A']
df_solar_dk2 = df_solar_dk2_a.merge(df_solar_dk2_fc1, left_on=df_solar_dk2_a['datetime_start_utc'], right_on=df_solar_dk2_fc1['datetime_start_utc'])

df_solar_dk1_fc1 = df_solar_dk1[df_solar_dk1.phase == 'FC1']
df_solar_dk1_a = df_solar_dk1[df_solar_dk1.phase == 'A']
df_solar_dk1 = df_solar_dk1_a.merge(df_solar_dk1_fc1, left_on=df_solar_dk1_a['datetime_start_utc'], right_on=df_solar_dk1_fc1['datetime_start_utc'])

df_solar_es_fc1 = df_solar_es[df_solar_es.phase == 'FC1']
df_solar_es_a = df_solar_es[df_solar_es.phase == 'A']
df_solar_es = df_solar_es_a.merge(df_solar_es_fc1, left_on=df_solar_es_a['datetime_start_utc'], right_on=df_solar_es_fc1['datetime_start_utc'])

df_solar_pl_fc1 = df_solar_pl[df_solar_pl.phase == 'FC1']
df_solar_pl_a = df_solar_pl[df_solar_pl.phase == 'A']
df_solar_pl = df_solar_pl_a.merge(df_solar_pl_fc1, left_on=df_solar_pl_a['datetime_start_utc'], right_on=df_solar_pl_fc1['datetime_start_utc'])

df_solar_ch_fc1 = df_solar_ch[df_solar_ch.phase == 'FC1']
df_solar_ch_a = df_solar_ch[df_solar_ch.phase == 'A']
df_solar_ch = df_solar_ch_a.merge(df_solar_ch_fc1, left_on=df_solar_ch_a['datetime_start_utc'], right_on=df_solar_ch_fc1['datetime_start_utc'])

df_solar_sk_fc1 = df_solar_sk[df_solar_sk.phase == 'FC1']
df_solar_sk_a = df_solar_sk[df_solar_sk.phase == 'A']
df_solar_sk = df_solar_sk_a.merge(df_solar_sk_fc1, left_on=df_solar_sk_a['datetime_start_utc'], right_on=df_solar_sk_fc1['datetime_start_utc'])

df_solar_itcs_fc1 = df_solar_itcs[df_solar_itcs.phase == 'FC1']
df_solar_itcs_a = df_solar_itcs[df_solar_itcs.phase == 'A']
df_solar_itcs = df_solar_itcs_a.merge(df_solar_itcs_fc1, left_on=df_solar_itcs_a['datetime_start_utc'], right_on=df_solar_itcs_fc1['datetime_start_utc'])

df_solar_itno_fc1 = df_solar_itno[df_solar_itno.phase == 'FC1']
df_solar_itno_a = df_solar_itno[df_solar_itno.phase == 'A']
df_solar_itno = df_solar_itno_a.merge(df_solar_itno_fc1, left_on=df_solar_itno_a['datetime_start_utc'], right_on=df_solar_itno_fc1['datetime_start_utc'])

df_solar_cz_fc1 = df_solar_cz[df_solar_cz.phase == 'FC1']
df_solar_cz_a = df_solar_cz[df_solar_cz.phase == 'A']
df_solar_cz = df_solar_cz_a.merge(df_solar_cz_fc1, left_on=df_solar_cz_a['datetime_start_utc'], right_on=df_solar_cz_fc1['datetime_start_utc'])

df_solar_si_fc1 = df_solar_si[df_solar_si.phase == 'FC1']
df_solar_si_a = df_solar_si[df_solar_si.phase == 'A']
df_solar_si = df_solar_si_a.merge(df_solar_si_fc1, left_on=df_solar_si_a['datetime_start_utc'], right_on=df_solar_si_fc1['datetime_start_utc'])

df_solar_be_elia_fc1 = df_solar_be_elia[df_solar_be_elia.phase == 'FC1']
df_solar_be_elia_a = df_solar_be_elia[df_solar_be_elia.phase == 'A']
df_solar_be_elia = df_solar_be_elia_a.merge(df_solar_be_elia_fc1, left_on=df_solar_be_elia_a['datetime_start_utc'], right_on=df_solar_be_elia_fc1['datetime_start_utc'])

df_solar_be_entsoeftp_fc1 = df_solar_be_entsoeftp[df_solar_be_entsoeftp.phase == 'FC1']
df_solar_be_entsoeftp_a = df_solar_be_entsoeftp[df_solar_be_entsoeftp.phase == 'A']
df_solar_be_entsoeftp = df_solar_be_entsoeftp_a.merge(df_solar_be_entsoeftp_fc1, left_on=df_solar_be_entsoeftp_a['datetime_start_utc'], right_on=df_solar_be_entsoeftp_fc1['datetime_start_utc'])

df_solar_at_entsoeftp_fc1 = df_solar_at_entsoeftp[df_solar_at_entsoeftp.phase == 'FC1']
df_solar_at_entsoeftp_a = df_solar_at_entsoeftp[df_solar_at_entsoeftp.phase == 'A']
df_solar_at_entsoeftp = df_solar_at_entsoeftp_a.merge(df_solar_at_entsoeftp_fc1, left_on=df_solar_at_entsoeftp_a['datetime_start_utc'], right_on=df_solar_at_entsoeftp_fc1['datetime_start_utc'])

df_solar_at_apg_fc1 = df_solar_at_apg[df_solar_at_apg.phase == 'FC1']
df_solar_at_apg_a = df_solar_at_apg[df_solar_at_apg.phase == 'A']
df_solar_at_apg = df_solar_at_apg_a.merge(df_solar_at_apg_fc1, left_on=df_solar_at_apg_a['datetime_start_utc'], right_on=df_solar_at_apg_fc1['datetime_start_utc'])

df_solar_fr_entsoeftp_fc1 = df_solar_fr_entsoeftp[df_solar_fr_entsoeftp.phase == 'FC1']
df_solar_fr_entsoeftp_a = df_solar_fr_entsoeftp[df_solar_fr_entsoeftp.phase == 'A']
df_solar_fr_entsoeftp = df_solar_fr_entsoeftp_a.merge(df_solar_fr_entsoeftp_fc1, left_on=df_solar_fr_entsoeftp_a['datetime_start_utc'], right_on=df_solar_fr_entsoeftp_fc1['datetime_start_utc'])

df_solar_fr_rte_fc1 = df_solar_fr_rte[df_solar_fr_rte.phase == 'FC1']
df_solar_fr_rte_a = df_solar_fr_rte[df_solar_fr_rte.phase == 'A']
df_solar_fr_rte = df_solar_fr_rte_a.merge(df_solar_fr_rte_fc1, left_on=df_solar_fr_rte_a['datetime_start_utc'], right_on=df_solar_fr_rte_fc1['datetime_start_utc'])

Het gebied GB heeft alleen maar records met Forecast data en geen Actual data. De uitkomst van de merge zou een lege tabel worden. We laten dit gebied buiten beschouwing aangezien ENTSOE_FTP de enige provider is die data levert voor het gebied GB. 

Verder hebben de gebieden NO2, NO1 en FI ook geen Actual data. Deze dataframes laten we ook buiten beschouwing. 

In de regio FR zijn er verschillende providers. Provider RTE_ECO2 levert ook geen Actual data. Voor deze provider in dit gebied laten we de data ook buiten beschouwing. 

###  Accuraatheid 
We berekenen de accuraatheid door de production_mw van de Forecast af te trekken met de production_mw van de Actual.

In [10]:
df_solar_hu['accuracy'] = df_solar_hu['production_mw_y'] - df_solar_hu['production_mw_x']
df_solar_nl['accuracy'] = df_solar_nl['production_mw_y'] - df_solar_nl['production_mw_x']
df_solar_de['accuracy'] = df_solar_de['production_mw_y'] - df_solar_de['production_mw_x']
df_solar_ro['accuracy'] = df_solar_ro['production_mw_y'] - df_solar_ro['production_mw_x']
df_solar_dk2['accuracy'] = df_solar_dk2['production_mw_y'] - df_solar_dk2['production_mw_x']
df_solar_dk1['accuracy'] = df_solar_dk1['production_mw_y'] - df_solar_dk1['production_mw_x']
df_solar_es['accuracy'] = df_solar_es['production_mw_y'] - df_solar_es['production_mw_x']
df_solar_pl['accuracy'] = df_solar_pl['production_mw_y'] - df_solar_pl['production_mw_x']
df_solar_ch['accuracy'] = df_solar_ch['production_mw_y'] - df_solar_ch['production_mw_x']
df_solar_sk['accuracy'] = df_solar_sk['production_mw_y'] - df_solar_sk['production_mw_x']
df_solar_itcs['accuracy'] = df_solar_itcs['production_mw_y'] - df_solar_itcs['production_mw_x']
df_solar_itno['accuracy'] = df_solar_itno['production_mw_y'] - df_solar_itno['production_mw_x']
df_solar_cz['accuracy'] = df_solar_cz['production_mw_y'] - df_solar_cz['production_mw_x']
df_solar_si['accuracy'] = df_solar_si['production_mw_y'] - df_solar_si['production_mw_x']
df_solar_be_elia['accuracy'] = df_solar_be_elia['production_mw_y'] - df_solar_be_elia['production_mw_x']
df_solar_be_entsoeftp['accuracy'] = df_solar_be_entsoeftp['production_mw_y'] - df_solar_be_entsoeftp['production_mw_x']
df_solar_at_entsoeftp['accuracy'] = df_solar_at_entsoeftp['production_mw_y'] - df_solar_at_entsoeftp['production_mw_x']
df_solar_at_apg['accuracy'] = df_solar_at_apg['production_mw_y'] - df_solar_at_apg['production_mw_x']
df_solar_fr_rte['accuracy'] = df_solar_fr_rte['production_mw_y'] - df_solar_fr_rte['production_mw_x']
df_solar_fr_entsoeftp['accuracy'] = df_solar_fr_entsoeftp['production_mw_y'] - df_solar_fr_entsoeftp['production_mw_x']


Dit zijn de gemiddeldes van de accuraatheid per areacode en provider. Deze gemiddeldes gebruiken we verder niet in het onderzoek.

In [11]:
gem_acc_hu = abs(df_solar_hu['accuracy']).mean()
gem_acc_nl = abs(df_solar_nl['accuracy']).mean()
gem_acc_de = abs(df_solar_de['accuracy']).mean()
gem_acc_ro = abs(df_solar_ro['accuracy']).mean()
gem_acc_dk2 = abs(df_solar_dk2['accuracy']).mean()
gem_acc_dk1 = abs(df_solar_dk1['accuracy']).mean()
gem_acc_es = abs(df_solar_es['accuracy']).mean()
gem_acc_pl = abs(df_solar_pl['accuracy']).mean()
gem_acc_ch = abs(df_solar_ch['accuracy']).mean()
gem_acc_sk = abs(df_solar_sk['accuracy']).mean()
gem_acc_itcs = abs(df_solar_itcs['accuracy']).mean()
gem_acc_itno = abs(df_solar_itno['accuracy']).mean()
gem_acc_cz = abs(df_solar_cz['accuracy']).mean()
gem_acc_si = abs(df_solar_si['accuracy']).mean()
gem_acc_be_elia = abs(df_solar_be_elia['accuracy']).mean()
gem_acc_be_entsoeftp = abs(df_solar_be_entsoeftp['accuracy']).mean()
gem_acc_at_entsoeftp = abs(df_solar_at_entsoeftp['accuracy']).mean()
gem_acc_at_apg = abs(df_solar_at_apg['accuracy']).mean()
gem_acc_fr_entsoeftp = abs(df_solar_fr_entsoeftp['accuracy']).mean()
gem_acc_fr_rte = abs(df_solar_fr_rte['accuracy']).mean()

In [12]:
lijst_gemiddelde_accuraatheid = [gem_acc_hu,gem_acc_nl,gem_acc_de,gem_acc_ro,gem_acc_dk2,gem_acc_dk1,gem_acc_es,gem_acc_pl,gem_acc_ch,gem_acc_sk,gem_acc_itcs,gem_acc_itno,gem_acc_cz,gem_acc_si,gem_acc_be_elia,gem_acc_be_entsoeftp,gem_acc_at_apg,gem_acc_at_entsoeftp,gem_acc_fr_rte,gem_acc_fr_entsoeftp]
lijst_gemiddelde_accuraatheid_namen = ['gem_acc_hu', 'gem_acc_nl', 'gem_acc_de', 'gem_acc_ro', 'gem_acc_dk2', 'gem_acc_dk1', 'gem_acc_es', 'gem_acc_pl', 'gem_acc_ch','gem_acc_sk','gem_acc_itcs','gem_acc_itno','gem_acc_cz','gem_acc_si','gem_acc_be_elia','gem_acc_be_entsoeftp','gem_acc_at_apg','gem_acc_at_entsoeftp','gem_acc_fr_rte','gem_acc_fr_entsoeftp']
res = "\n".join("{} : {}".format(naam, waarde) for naam, waarde in zip(lijst_gemiddelde_accuraatheid_namen, lijst_gemiddelde_accuraatheid))
print(res)

gem_acc_hu : 28.157512787235664
gem_acc_nl : 750.7983503580537
gem_acc_de : 376.5452130850033
gem_acc_ro : 36.63570847818773
gem_acc_dk2 : 10.458119432327825
gem_acc_dk1 : 21.368465709532316
gem_acc_es : 216.4998656242365
gem_acc_pl : 96.9901710319164
gem_acc_ch : 251.01176211453745
gem_acc_sk : 18.10932867818204
gem_acc_itcs : 65.422410911279
gem_acc_itno : 110.35254395935014
gem_acc_cz : 26.543488670371957
gem_acc_si : 2.231537941212643
gem_acc_be_elia : 77.61333058809176
gem_acc_be_entsoeftp : 72.76242289134551
gem_acc_at_apg : 134.19325842696628
gem_acc_at_entsoeftp : 106.79447871495755
gem_acc_fr_rte : 415.07639791937584
gem_acc_fr_entsoeftp : 415.11835263516446


Met het VWMAPE berekenen we de accuraahteid per area_code en provider door het totale absolute verschil tussen de FC1 en A te delen door het totale A.

In [16]:
hu_accuracy_per = sum(abs(df_solar_hu['accuracy'])) / sum(df_solar_hu['production_mw_x']) * 100
nl_accuracy_per = sum(abs(df_solar_nl['accuracy'])) / sum(df_solar_nl['production_mw_x']) * 100
de_accuracy_per = sum(abs(df_solar_de['accuracy'])) / sum(df_solar_de['production_mw_x']) * 100
ro_accuracy_per = sum(abs(df_solar_ro['accuracy'])) / sum(df_solar_ro['production_mw_x']) * 100
dk2_accuracy_per = sum(abs(df_solar_dk2['accuracy'])) / sum(df_solar_dk2['production_mw_x']) * 100
dk1_accuracy_per = sum(abs(df_solar_dk1['accuracy'])) / sum(df_solar_dk1['production_mw_x']) * 100
es_accuracy_per = sum(abs(df_solar_es['accuracy'])) / sum(df_solar_es['production_mw_x']) * 100
pl_accuracy_per = sum(abs(df_solar_pl['accuracy'])) / sum(df_solar_pl['production_mw_x']) * 100
ch_accuracy_per = sum(abs(df_solar_ch['accuracy'])) / sum(df_solar_ch['production_mw_x']) * 100
sk_accuracy_per = sum(abs(df_solar_sk['accuracy'])) / sum(df_solar_sk['production_mw_x']) * 100
itcs_accuracy_per = sum(abs(df_solar_itcs['accuracy'])) / sum(df_solar_itcs['production_mw_x']) * 100
itno_accuracy_per = sum(abs(df_solar_itno['accuracy'])) / sum(df_solar_itno['production_mw_x']) * 100
cz_accuracy_per = sum(abs(df_solar_cz['accuracy'])) / sum(df_solar_cz['production_mw_x']) * 100
si_accuracy_per = sum(abs(df_solar_si['accuracy'])) / sum(df_solar_si['production_mw_x']) * 100
be_elia_accuracy_per = sum(abs(df_solar_be_elia['accuracy'])) / sum(df_solar_be_elia['production_mw_x']) * 100
be_entsoeftp_accuracy_per = sum(abs(df_solar_be_entsoeftp['accuracy'])) / sum(df_solar_be_entsoeftp['production_mw_x']) * 100
at_apg_accuracy_per = sum(abs(df_solar_at_apg['accuracy'])) / sum(df_solar_at_apg['production_mw_x']) * 100
at_entsoeftp_accuracy_per = sum(abs(df_solar_at_entsoeftp['accuracy'])) / sum(df_solar_at_entsoeftp['production_mw_x']) * 100
fr_entsoeftp_accuracy_per = sum(abs(df_solar_fr_entsoeftp['accuracy'])) / sum(df_solar_fr_entsoeftp['production_mw_x']) * 100
fr_rte_accuracy_per = sum(abs(df_solar_fr_rte['accuracy'])) / sum(df_solar_fr_rte['production_mw_x']) * 100


## Samengevoegde dataframe
Om in PowerBI te laden. Onnodige kolommen worden verwijderd om voor minder laadtijd te zorgen. 

In [17]:
df_samengevoegd = pd.concat([df_solar_hu, df_solar_nl, df_solar_de, df_solar_ro, df_solar_dk2, df_solar_dk1, df_solar_es, df_solar_pl, df_solar_ch, df_solar_sk, df_solar_itcs, df_solar_itno, df_solar_cz, df_solar_si, df_solar_be_elia, df_solar_be_entsoeftp, df_solar_at_entsoeftp, df_solar_at_apg, df_solar_fr_rte, df_solar_fr_entsoeftp])

In [19]:
df_samengevoegd = df_samengevoegd.drop(['key_0', 
                                        'datetime_start_utc_y', 
                                        'datetime_end_utc_y', 
                                        'provider_y', 
                                        'area_code_y', 
                                        'interval_y', 
                                        'phase_y',], axis=1)

# Betrouwbaarheid
De betrouwbaarheid berekenen we door het totaal aantal keer dat een actual/forecast voorspeld had moeten zijn (laatste dag - eerste dag). Dit wordt gedeeld door de daadwerkelijke hoeveelheid actual/forecast die de provider heeft gegeven, * 100%. 

In [20]:
# hu: interval = 15
hu_verschil = (df_solar_hu['datetime_start_utc_x'].iloc[-1]) - (df_solar_hu['datetime_start_utc_x'].iloc[-0])
hu_uren = hu_verschil.days*24 + hu_verschil.seconds/3600
hu_kwartiertjes = hu_uren*4
hu_rijen = len(df_solar_hu)
hu_betrouwbaarheid = (hu_rijen / hu_kwartiertjes)*100

# nl: interval = 15
nl_verschil = (df_solar_nl['datetime_start_utc_x'].iloc[-1]) - (df_solar_nl['datetime_start_utc_x'].iloc[-0])
nl_uren = nl_verschil.days*24 + nl_verschil.seconds/3600
nl_kwartiertjes = nl_uren*4
nl_rijen = len(df_solar_nl)
nl_betrouwbaarheid = (nl_rijen / nl_kwartiertjes)*100

# de: interval = 15
de_verschil = (df_solar_de['datetime_start_utc_x'].iloc[-1]) - (df_solar_de['datetime_start_utc_x'].iloc[-0])
de_uren = de_verschil.days*24 + de_verschil.seconds/3600
de_kwartiertjes = de_uren*4
de_rijen = len(df_solar_de)
de_betrouwbaarheid = (de_rijen / de_kwartiertjes)*100

# ro: interval 15=26141, 60=9527
ro_verschil = (df_solar_ro['datetime_start_utc_x'].iloc[-1]) - (df_solar_ro['datetime_start_utc_x'].iloc[0])
ro_uren = ro_verschil.days*24 + ro_verschil.seconds/3600
ro_kwartiertjes = ro_uren*4
ro_rijen = 26141 + (9527*4)
ro_betrouwbaarheid = (ro_rijen/ro_kwartiertjes)*100

# dk1 interval=60
dk1_verschil = (df_solar_dk1['datetime_start_utc_x'].iloc[-1]) - (df_solar_dk1['datetime_start_utc_x'].iloc[0])
dk1_uren = dk1_verschil.days*24 + dk1_verschil.seconds/3600
dk1_rijen = len(df_solar_dk1)
dk1_betrouwbaarheid = (dk1_rijen / dk1_uren)*100

#dk2 interval=60
dk2_verschil = (df_solar_dk2['datetime_start_utc_x'].iloc[-1]) - (df_solar_dk2['datetime_start_utc_x'].iloc[0])
dk2_uren = dk2_verschil.days*24 + dk2_verschil.seconds/3600
dk2_rijen = len(df_solar_dk2)
dk2_betrouwbaarheid = (dk2_rijen / dk2_uren)*100

# es interval=60
es_verschil = (df_solar_es['datetime_start_utc_x'].iloc[-1]) - (df_solar_es['datetime_start_utc_x'].iloc[0])
es_uren = es_verschil.days*24 + es_verschil.seconds/3600
es_rijen = len(df_solar_es)
es_betrouwbaarheid = (es_rijen / es_uren)*100

# pl interval=60
pl_verschil = (df_solar_pl['datetime_start_utc_x'].iloc[-1]) - (df_solar_pl['datetime_start_utc_x'].iloc[0])
pl_uren = pl_verschil.days*24 + pl_verschil.seconds/3600
pl_rijen = len(df_solar_pl)
pl_betrouwbaarheid = (pl_rijen / pl_uren)*100

# ch interval=60
ch_verschil = (df_solar_ch['datetime_start_utc_x'].iloc[-1]) - (df_solar_ch['datetime_start_utc_x'].iloc[0])
ch_uren = ch_verschil.days*24 + ch_verschil.seconds/3600
ch_rijen = len(df_solar_ch)
ch_betrouwbaarheid = (ch_rijen / ch_uren)*100

# sk interval=60
sk_verschil = (df_solar_sk['datetime_start_utc_x'].iloc[-1]) - (df_solar_sk['datetime_start_utc_x'].iloc[0])
sk_uren = sk_verschil.days*24 + sk_verschil.seconds/3600
sk_rijen = len(df_solar_sk)
sk_betrouwbaarheid = (sk_rijen / sk_uren)*100

# itcs interval=60
itcs_verschil = (df_solar_itcs['datetime_start_utc_x'].iloc[-1]) - (df_solar_itcs['datetime_start_utc_x'].iloc[0])
itcs_uren = itcs_verschil.days*24 + itcs_verschil.seconds/3600
itcs_rijen = len(df_solar_itcs)
itcs_betrouwbaarheid = (itcs_rijen / itcs_uren)*100

# itno interval=60
itno_verschil = (df_solar_itno['datetime_start_utc_x'].iloc[-1]) - (df_solar_itno['datetime_start_utc_x'].iloc[0])
itno_uren = itno_verschil.days*24 + itno_verschil.seconds/3600
itno_rijen = len(df_solar_itno)
itno_betrouwbaarheid = (itno_rijen / itno_uren)*100

# cz interval=60
cz_verschil = (df_solar_cz['datetime_start_utc_x'].iloc[-1]) - (df_solar_cz['datetime_start_utc_x'].iloc[0])
cz_uren = cz_verschil.days*24 + cz_verschil.seconds/3600
cz_rijen = len(df_solar_cz)
cz_betrouwbaarheid = (cz_rijen / cz_uren)*100

# si interval=60
si_verschil = (df_solar_si['datetime_start_utc_x'].iloc[-1]) - (df_solar_si['datetime_start_utc_x'].iloc[0])
si_uren = si_verschil.days*24 + si_verschil.seconds/3600
si_rijen = len(df_solar_si)
si_betrouwbaarheid = (si_rijen / si_uren)*100

# be elia interval=15
be_elia_verschil = (df_solar_be_elia['datetime_start_utc_x'].iloc[-1]) - (df_solar_be_elia['datetime_start_utc_x'].iloc[-0])
be_elia_uren = be_elia_verschil.days*24 + be_elia_verschil.seconds/3600
be_elia_kwartiertjes = be_elia_uren*4
be_elia_rijen = len(df_solar_be_elia)
be_elia_betrouwbaarheid = (be_elia_rijen / be_elia_kwartiertjes)*100

# be entsoeftp interval=60
be_entsoeftp_verschil = (df_solar_be_entsoeftp['datetime_start_utc_x'].iloc[-1]) - (df_solar_be_entsoeftp['datetime_start_utc_x'].iloc[0])
be_entsoeftp_uren = be_entsoeftp_verschil.days*24 + be_entsoeftp_verschil.seconds/3600
be_entsoeftp_rijen = len(df_solar_be_entsoeftp)
be_entsoeftp_betrouwbaarheid = (be_entsoeftp_rijen / be_entsoeftp_uren)*100

# at apg interval=15
at_apg_verschil = (df_solar_at_apg['datetime_start_utc_x'].iloc[-1]) - (df_solar_at_apg['datetime_start_utc_x'].iloc[-0])
at_apg_uren = at_apg_verschil.days*24 + at_apg_verschil.seconds/3600
at_apg_kwartiertjes = at_apg_uren*4
at_apg_rijen = len(df_solar_at_apg)
at_apg_betrouwbaarheid = (at_apg_rijen / at_apg_kwartiertjes)*100

# at entsoeftp interval=15
at_entsoeftp_verschil = (df_solar_at_entsoeftp['datetime_start_utc_x'].iloc[-1]) - (df_solar_at_entsoeftp['datetime_start_utc_x'].iloc[-0])
at_entsoeftp_uren = at_entsoeftp_verschil.days*24 + at_entsoeftp_verschil.seconds/3600
at_entsoeftp_kwartiertjes = at_entsoeftp_uren*4
at_entsoeftp_rijen = len(df_solar_at_entsoeftp)
at_entsoeftp_betrouwbaarheid = (at_entsoeftp_rijen / at_entsoeftp_kwartiertjes)*100

# fr rte interval=60
fr_rte_verschil = (df_solar_fr_rte['datetime_start_utc_x'].iloc[-1]) - (df_solar_fr_rte['datetime_start_utc_x'].iloc[0])
fr_rte_uren = fr_rte_verschil.days*24 + fr_rte_verschil.seconds/3600
fr_rte_rijen = len(df_solar_fr_rte)
fr_rte_betrouwbaarheid = (fr_rte_rijen / fr_rte_uren)*100

#fr entsoeftp interval=60
fr_entsoeftp_verschil = (df_solar_fr_entsoeftp['datetime_start_utc_x'].iloc[-1]) - (df_solar_fr_entsoeftp['datetime_start_utc_x'].iloc[0])
fr_entsoeftp_uren = fr_entsoeftp_verschil.days*24 + fr_entsoeftp_verschil.seconds/3600
fr_entsoeftp_rijen = len(df_solar_fr_entsoeftp)
fr_entsoeftp_betrouwbaarheid = (fr_entsoeftp_rijen / fr_entsoeftp_uren)*100

In [21]:
x = [
    hu_betrouwbaarheid,
    nl_betrouwbaarheid,
    de_betrouwbaarheid,
    ro_betrouwbaarheid,
    dk1_betrouwbaarheid,
    dk2_betrouwbaarheid,
    es_betrouwbaarheid,
    pl_betrouwbaarheid,
    ch_betrouwbaarheid,
    sk_betrouwbaarheid,
    itcs_betrouwbaarheid,
    itno_betrouwbaarheid,
    cz_betrouwbaarheid,
    si_betrouwbaarheid,
    be_elia_betrouwbaarheid,
    be_entsoeftp_betrouwbaarheid,
    fr_rte_betrouwbaarheid,
    fr_entsoeftp_betrouwbaarheid,
    at_entsoeftp_betrouwbaarheid,
    at_apg_betrouwbaarheid
]

## Snelheid
De snelheid berekenen we door het verschil tussen datetime_start_utc en datetime_modification_utc te nemen. We willen weten hoe snel de forecast is, waardoor we niet kijken naar de records met A als phase. De datetime_modification_utc zou eerder dan de datetime_start_utc moeten zijn, echter zien wij dat het in veel gevallen niet zo is. Deze records nemen we niet mee in het onderzoek. 

In [22]:
df['Difference'] = (df['datetime_start_utc'] - df['datetime_modification_utc']).dt.total_seconds()

In [36]:
df_difference = df[df.Difference > 0]
df_difference = df_difference[df_difference.phase == 'FC1']
difference = df_difference.groupby('provider')

In [25]:
df_difference_fr = df_difference[df_difference.area_code== 'FR']
difference_fr = df_difference_fr.groupby('provider')

In [29]:
lijst_accuraatheid_per = [
    hu_accuracy_per,
    nl_accuracy_per,
    de_accuracy_per,
    ro_accuracy_per,
    dk2_accuracy_per,
    dk1_accuracy_per,
    es_accuracy_per,
    pl_accuracy_per,
    ch_accuracy_per,
    sk_accuracy_per,
    itcs_accuracy_per,
    itno_accuracy_per,
    cz_accuracy_per,
    si_accuracy_per,
    be_elia_accuracy_per,
    be_entsoeftp_accuracy_per,   
    fr_rte_accuracy_per,
    fr_entsoeftp_accuracy_per,
    at_entsoeftp_accuracy_per,
    at_apg_accuracy_per
]

In [30]:
lijst_landen = [
    'hu',
    'nl',
    'de',
    'ro',
    'dk2',
    'dk1',
    'es',
    'pl',
    'ch',
    'sk',
    'itcs',
    'itno',
    'cz',
    'si',
    'be',
    'be',
    'fr',
    'fr',
    'at',
    'at'
]

In [31]:
lijst_provider = [
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'ELIA',
    'ENTSOE_FTP',
    'RTE',
    'ENTSOE_FTP',
    'ENTSOE_FTP',
    'APG'
]

In [32]:
dfScores = pd.DataFrame(lijst_landen, columns =['area_code'])
dfScores['provider'] = pd.DataFrame(lijst_provider)
dfScores['reliability'] = pd.DataFrame(x)
dfScores['accuracy'] = pd.DataFrame(lijst_accuraatheid_per)